In [4]:
from git import Repo
import subprocess
import networkx as nx
import matplotlib as mpl
import matplotlib.cm as cm

EMPTY_TREE_SHA = "4b825dc642cb6eb9a060e54bf8d69288fbee4904"

def add_edge(G, sp, ep, fname):
    if ep in G[sp]:
        G[sp][ep]['files'].append(fname)
    else:
        G.add_edge(sp, ep, files=[fname])

def build_commit_graph(repo):
    ctype_map = {'A': 'D', 'D': 'A', 'M': 'M', 'R': 'R'}
    
    commits = list(repo.iter_commits())
    G = nx.DiGraph()
    f_to_c = {}
    for commit in reversed(commits):
        # add commit to graph
        sha = commit.hexsha
        G.add_node(sha)
        
        if not commit.parents:
            diff_index = commit.diff(EMPTY_TREE_SHA, create_patch=True)
        else:
            # parents[0] is the commit that was merged into
            diff_index = commit.diff(commit.parents[0], create_patch=True)
            
        # handle create_patch=True, also fix a GitPython bug (R100), see below for details
        # https://github.com/gitpython-developers/GitPython/issues/563
        for diff in diff_index:
            if diff.new_file:
                diff.change_type = 'A'
            elif diff.deleted_file:
                diff.change_type = 'D'
            elif diff.renamed: 
                diff.change_type = 'R'
            elif diff.a_blob and diff.b_blob and diff.a_blob != diff.b_blob:
                diff.change_type = 'M'
            else:
                raise Exception('Non-existent Change Type!')
            
        # iterate over changes
        print("{}: {}".format(sha, " ".join([ctype_map[diff.change_type] for diff in diff_index])))
        for diff in diff_index:
            if diff.change_type == ctype_map['A']:
                fname = diff.a_blob.path
                f_to_c[fname] = sha
            elif diff.change_type == ctype_map['D']:
                fname = diff.b_blob.path
                add_edge(G, sha, f_to_c[fname], fname)
                del f_to_c[fname]
            elif diff.change_type == ctype_map['R']:
                # old_fname = diff.b_blob.path
                # new_fname = diff.a_blob.path
                # when similarity is 100%, diff.a_blob and diff.b_blob are None
                old_fname = diff.rename_to
                new_fname = diff.rename_from
                assert old_fname in f_to_c
                add_edge(G, sha, f_to_c[old_fname], "{} -> {}".format(old_fname, new_fname))
                del f_to_c[old_fname]
                f_to_c[new_fname] = sha
            else:
                # change_type 'M'
                assert diff.b_blob.path == diff.a_blob.path
                fname = diff.b_blob.path
                add_edge(G, sha, f_to_c[fname], fname)
                f_to_c[fname] = sha
    return G

def map_to_color(x, m):
    color_list = [int(255 * c) for c in m.to_rgba(x)]
    return '#%02x%02x%02x%02x' % tuple(color_list)
        
def write_G_to_dot_with_pr(G, pr, fname):
    norm = mpl.colors.Normalize(vmin=min(pr.values()), vmax=max(pr.values()))
    cmap = cm.Blues
    m = cm.ScalarMappable(norm, cmap=cmap)
    with open(fname, 'w+') as f:
        f.write('digraph graphname {\n')
        for n in G.nodes_iter(data=False):
            color_str = map_to_color(pr[n], m)
            f.write('\"{}\" [style=filled fillcolor="{}" tooltip=\"{}\"];\n'.format(n, color_str, pr[n]))
        for e in G.edges_iter(data=True):
            f.write('\"{}\" -> \"{}\" [ label=\"{}\"];\n'.format(e[0], e[1], '&#10;'.join(e[2]['files'][:3])))
        
        f.write('}')
        
def draw_commit_graph(repo_name):
    repo = Repo('./' + repo_name)
    G = build_commit_graph(repo)
    pr = nx.pagerank(G, alpha=0.85)
    write_G_to_dot_with_pr(G, pr, repo_name + ".dot")
    subprocess.call('dot -Tsvg {}.dot -o {}.svg'.format(repo_name, repo_name), shell=True)

In [5]:
draw_commit_graph('git-playground')

09ac13e7898a95ff040852a4eb8914fea0b77112: A
eb0791ddaa0b12e701a5ad781c2fe91296d404d9: R A
a8eb2e44a4227d10d0f25bc3329466bd4b6992ab: M A
d9e049b2ff4e8da0575fd83295091141dba1c840: D A
2bbe0aa7322a7360003ba06130a6815be34e49f0: D A
3d341d435b5ceee208858d247d7507e68270aae2: R


In [ ]:
draw_commit_graph('Sexain-MemController')

In [ ]:
"""
diff --git a/file1.txt b/file1.txt
deleted file mode 100644
index be2348c3d866b914c3a6e7d6c555bb5d7bd304bc..0000000000000000000000000000000000000000
--- a/file1.txt
+++ /dev/null
@@ -1 +0,0 @@
-random stuff in file1

diff --git a/file3.txt b/file1.txt
similarity index 100%
rename from file3.txt
rename to file1.txt
diff --git a/file2.txt b/file2.txt
deleted file mode 100644
index 418e1293379426f4cbbac7faec06d10783e433dd..0000000000000000000000000000000000000000
--- a/file2.txt
+++ /dev/null
@@ -1 +0,0 @@
-random stuff in file2.txt

diff --git a/file3.txt b/file3.txt
index c823b19ce6a339cd8245783669a0c0d55bd94528..be2348c3d866b914c3a6e7d6c555bb5d7bd304bc 100644
--- a/file3.txt
+++ b/file3.txt
@@ -1 +1 @@
-random stuff in file3.txt
+random stuff in file1
diff --git a/file4.txt b/file4.txt
deleted file mode 100644
index 06c33614449231211067102407f59cdfaaecd969..0000000000000000000000000000000000000000
--- a/file4.txt
+++ /dev/null
@@ -1 +0,0 @@
-some random stuff in file4.txt

diff --git a/file2.txt b/file2.txt
new file mode 100644
index 0000000000000000000000000000000000000000..418e1293379426f4cbbac7faec06d10783e433dd
--- /dev/null
+++ b/file2.txt
@@ -0,0 +1 @@
+random stuff in file2.txt
diff --git a/file5.txt b/file5.txt
deleted file mode 100644
index e69de29bb2d1d6434b8b29ae775ad8c2e48c5391..0000000000000000000000000000000000000000

diff --git a/file4.txt b/file4.txt
new file mode 100644
index 0000000000000000000000000000000000000000..06c33614449231211067102407f59cdfaaecd969
--- /dev/null
+++ b/file4.txt
@@ -0,0 +1 @@
+some random stuff in file4.txt
diff --git a/file6.txt b/file6.txt
deleted file mode 100644
index 7572d73dde61720d2c8742e4af924ad7a1bb1d18..0000000000000000000000000000000000000000
--- a/file6.txt
+++ /dev/null
@@ -1,2 +0,0 @@
-random stuff in file4.txt
-this file will be renamed to file6.txt in this commit

diff --git a/file7.txt b/file6.txt
similarity index 97%
rename from file7.txt
rename to file6.txt
index 92f420a9cbfaa9be9966a12abb312f0d5d7c4423..7572d73dde61720d2c8742e4af924ad7a1bb1d18 100644
--- a/file7.txt
+++ b/file6.txt
@@ -1,3 +1,2 @@
 random stuff in file4.txt
 this file will be renamed to file6.txt in this commit
-a

"""
# print(text.decode('utf-8')) 
# line 415 in /Users/Beaver/anaconda3/lib/python3.5/site-packages/git/diff.py